In [ ]:
"""
# list all measurements, with buses and equipments - DistMeasurement
PREFIX r: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c: <http://iec.ch/TC57/CIM100#>
SELECT ?class ?type ?name ?bus ?phases ?eqtype ?eqname ?eqid ?trmid ?id WHERE {
#VALUES ?fdrid {"_77966920-E1EC-EE8A-23EE-4EFD23B205BD"}
#VALUES ?fdrid {"_67AB291F-DCCD-31B7-B499-338206B9828F"}
#VALUES ?fdrid {"_5B816B93-7A5F-B64C-8460-47C17D6E4B0F"}
#VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}
#VALUES ?fdrid {"_49003F52-A359-C2EA-10C4-F4ED3FD368CC"}
#VALUES ?fdrid {"_C1C3E687-6FFD-C753-582B-632A27E28507"}
#VALUES ?fdrid {"_E407CBB6-8C8D-9BC9-589C-AB83FBF0826D"}
#VALUES ?fdrid {"_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"}
#VALUES ?fdrid {"_AAE94E4A-2465-6F5E-37B1-3E72183A4E44"}
#VALUES ?fdrid {"_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"}
#VALUES ?fdrid {"_503D6E20-F499-4CC7-8051-971E23D0BF79"}
 ?eq c:Equipment.EquipmentContainer ?fdr.
 ?fdr c:IdentifiedObject.mRID ?fdrid. 
{ ?s r:type c:Discrete. bind ("Discrete" as ?class)}
  UNION
{ ?s r:type c:Analog. bind ("Analog" as ?class)}
 ?s c:IdentifiedObject.name ?name .
 ?s c:IdentifiedObject.mRID ?id .
 ?s c:Measurement.PowerSystemResource ?eq .
 ?s c:Measurement.Terminal ?trm .
 ?s c:Measurement.measurementType ?type .
 ?trm c:IdentifiedObject.mRID ?trmid.
 ?eq c:IdentifiedObject.mRID ?eqid.
 ?eq c:IdentifiedObject.name ?eqname.
 ?eq r:type ?typeraw.
  bind(strafter(str(?typeraw),"#") as ?eqtype)
 ?trm c:Terminal.ConnectivityNode ?cn.
 ?cn c:IdentifiedObject.name ?bus.
 ?s c:Measurement.phases ?phsraw .
   {bind(strafter(str(?phsraw),"PhaseCode.") as ?phases)}
} ORDER BY ?class ?type ?name
"""

## Measurements for Transformers

In [ ]:
"""
# list measurement points for PowerTransformer with tanks
PREFIX r: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c: <http://iec.ch/TC57/CIM100#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?name ?wnum ?bus ?phs ?eqid ?trmid WHERE { 
 VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}
 ?s c:Equipment.EquipmentContainer ?fdr.
 ?fdr c:IdentifiedObject.mRID ?fdrid. 
 ?s r:type c:PowerTransformer.
 ?end c:TransformerTankEnd.TransformerTank ?tank.
 ?tank c:TransformerTank.PowerTransformer ?s.
 ?s c:IdentifiedObject.name ?name.
 ?s c:IdentifiedObject.mRID ?eqid.
 ?end c:TransformerEnd.Terminal ?trm.
 ?end c:TransformerEnd.endNumber ?wnum.
 ?trm c:IdentifiedObject.mRID ?trmid. 
 ?trm c:Terminal.ConnectivityNode ?cn. 
 ?cn c:IdentifiedObject.name ?bus.
 OPTIONAL {?end c:TransformerTankEnd.phases ?phsraw.
  bind(strafter(str(?phsraw),"PhaseCode.") as ?phs)}
}
ORDER BY ?name ?wnum ?phs
"""

In [ ]:
"""
# list measurement points for PowerTransformer with no tanks
PREFIX r: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c: <http://iec.ch/TC57/CIM100#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?name ?wnum ?bus ?eqid ?trmid WHERE { 
 VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}
 ?s c:Equipment.EquipmentContainer ?fdr.
 ?fdr c:IdentifiedObject.mRID ?fdrid. 
 ?s r:type c:PowerTransformer.
 ?s c:IdentifiedObject.name ?name.
 ?s c:IdentifiedObject.mRID ?eqid.
 ?end c:PowerTransformerEnd.PowerTransformer ?s.
 ?end c:TransformerEnd.Terminal ?trm.
 ?end c:TransformerEnd.endNumber ?wnum.
 ?trm c:IdentifiedObject.mRID ?trmid. 
 ?trm c:Terminal.ConnectivityNode ?cn. 
 ?cn c:IdentifiedObject.name ?bus.
}
ORDER BY ?name ?wnum
"""

## Measurements for DERs

In [ ]:
"""

# list measurement points for PowerElectronicsConnection with BatteryUnit in a selected feeder
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?name ?uname ?bus ?id (group_concat(distinct ?phs;separator="") as ?phases) WHERE {
  SELECT ?name ?uname ?bus ?phs ?id WHERE {
 VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}  # 13 bus
 ?fdr c:IdentifiedObject.mRID ?fdrid.
 ?s r:type c:PowerElectronicsConnection.
 ?s c:Equipment.EquipmentContainer ?fdr.
 ?s c:IdentifiedObject.name ?name.
 ?s c:IdentifiedObject.mRID ?id.
 ?peu r:type c:BatteryUnit.
 ?peu c:IdentifiedObject.name ?uname.
 ?s c:PowerElectronicsConnection.PowerElectronicsUnit ?peu.
 ?t1 c:Terminal.ConductingEquipment ?s.
 ?t1 c:ACDCTerminal.sequenceNumber "1".
 ?t1 c:Terminal.ConnectivityNode ?cn1. 
 ?cn1 c:IdentifiedObject.name ?bus.
	OPTIONAL {?pep c:PowerElectronicsConnectionPhase.PowerElectronicsConnection ?s.
 	?pep c:PowerElectronicsConnectionPhase.phase ?phsraw.
   	bind(strafter(str(?phsraw),"SinglePhaseKind.") as ?phs) }
 } ORDER BY ?name ?phs
}
GROUP BY ?name ?uname ?bus ?id
ORDER BY ?name
"""

In [ ]:
"""
# Get all measurements associated with inverters (PowerElectronicsConnection)
# NOTE: It would appear that the measurements for inverters are
# associated with PowerElectronicsConnection objects and NOT
# PowerElectronicsConnectionPhase objects. You can prove this by
# replacing PowerElectronicsConnection with
# PowerElectronicsConnectionPhase below.
PREFIX r: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c: <http://iec.ch/TC57/CIM100#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?inverter_mrid ?meas_mrid ?phase
WHERE {
  # Update for your feeder, or remove for all feeders.
  VALUES ?feeder_mrid {"_AAE94E4A-2465-6F5E-37B1-3E72183A4E44"}
  ?s r:type ?type.
  ?s c:IdentifiedObject.mRID ?meas_mrid.
  ?s c:Measurement.PowerSystemResource ?eq.
  ?s c:Measurement.Terminal ?trm.
  ?s c:Measurement.phases ?phsraw.
  {bind(strafter(str(?phsraw),"PhaseCode.") as ?phase)} .
  ?eq c:IdentifiedObject.mRID ?inverter_mrid.
  ?eq r:type c:PowerElectronicsConnection.
  ?eq c:Equipment.EquipmentContainer ?fdr.
  ?fdr c:IdentifiedObject.mRID ?feeder_mrid.
}
ORDER BY ?inverter_mrid
"""

## Measurements for Lines

In [ ]:
"""
# list measurement points for ACLineSegments in a selected feeder
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?name ?bus1 ?bus2 ?id (group_concat(distinct ?phs;separator="") as ?phases) WHERE {
  SELECT ?name ?bus1 ?bus2 ?phs ?id WHERE {
 VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}  # 13 bus
 ?fdr c:IdentifiedObject.mRID ?fdrid.
 ?s r:type c:ACLineSegment.
 ?s c:Equipment.EquipmentContainer ?fdr.
 ?s c:IdentifiedObject.name ?name.
 ?s c:IdentifiedObject.mRID ?id.
 ?t1 c:Terminal.ConductingEquipment ?s.
 ?t1 c:ACDCTerminal.sequenceNumber "1".
 ?t1 c:Terminal.ConnectivityNode ?cn1. 
 ?cn1 c:IdentifiedObject.name ?bus1.
 ?t2 c:Terminal.ConductingEquipment ?s.
 ?t2 c:ACDCTerminal.sequenceNumber "2".
 ?t2 c:Terminal.ConnectivityNode ?cn2. 
 ?cn2 c:IdentifiedObject.name ?bus2
	OPTIONAL {?acp c:ACLineSegmentPhase.ACLineSegment ?s.
 	?acp c:ACLineSegmentPhase.phase ?phsraw.
   	bind(strafter(str(?phsraw),"SinglePhaseKind.") as ?phs) }
 } ORDER BY ?name ?phs
}
GROUP BY ?name ?bus1 ?bus2 ?id
ORDER BY ?name
"""